Install Langchain Python Package and Python 3.10+

In [20]:
from langchain.embeddings import GPT4AllEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain, PromptTemplate
from langchain.prompts import PromptTemplate

# Import PyPDF 
from PyPDF2 import PdfReader

# Import Gradio Tool
import gradio as gr

import os

In [3]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

In [8]:
llm = ChatOpenAI(temperature = 0.7)

In [3]:
# llm("What can you tell me about Crypto?")

In [5]:
loader = PyPDFLoader("./2023_ASQ_PRQC_Presentation/risk_management_template.pdf")
pages = loader.load_and_split()

# text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(pages)

In [6]:
vectordb = FAISS.from_documents(pages, embedding=GPT4AllEmbeddings())

In [33]:
def rmr_writer(task, history):
    template_path = "./2023_ASQ_PRQC_Presentation/risk_management_template.pdf"

    pdf_reader = PdfReader(template_path)
    pdf_text = ""
    for i, page in enumerate(pdf_reader.pages):
        # print(page.extract_text())
        pdf_text += page.extract_text() + "\n"

    # Define prompt
    prompt_text = """You are a quality engineer who is responsible for writing the risk management report for a new product. 
    You must have attention to detail and follow the template provided by the company.
    You will be given a task to complete and a template to use for each task. 
    
    Complete this task:
    {task}

    Using this template:
    {pdf_text}"""

    prompt = PromptTemplate.from_template(prompt_text)

    prompt.format(pdf_text=pdf_text, task=task)

    # Define LLM chain
    llm = ChatOpenAI(model = "gpt-3.5-turbo-16k",temperature=0.7)
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    return llm_chain.run(pdf_text = pdf_text, task=task)

In [34]:
app_writer = gr.ChatInterface(rmr_writer)

app_writer.launch(share=False)

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


write a risk management report about a bone cement called adhesibone

In [9]:
def question2pdf(message, history):
    qachain=RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())
    return qachain({"query": message})['result']

app_writer = gr.ChatInterface(question2pdf)

app_writer.launch(share=False)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://53bda8ea24e88ccff5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [35]:
app_writer.close()